In [131]:
# Import the necessary libraries

# import sql libraries
import pyodbc

# import data manipulation libraries
import pandas as pd
import os

# import data visualization libraries
import plotly_express as px
import plotly.graph_objs as go
import plotly as py


#import parallel processing libraries
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
import multiprocessing as mp

# import machine learning libraries
from sklearn.cluster import KMeans

In [47]:
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\\Users\\33646\\Desktop\\M2\\EEE\\DataBase\\Project\\DatabaseProjet.accdb;')


In [52]:
sql = 'SELECT SA.ActivityName AS Name_of_Activity, S.SessionId AS ID_Session, I.FirstName & " " & I.LastName AS Animators FROM Inhabitant AS I INNER JOIN (SkillActivity AS SA INNER JOIN [Session] AS S ON SA.ActivityName = S.ActivityName) ON I.InHabitantId = S.InHabitantId;'

In [50]:
df = pd.read_sql(sql, conn)

c:\Users\33646\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'SELECT SA.ActivityName AS Name_of_Activity, S.SessionId AS ID_Session, I.FirstName & " " & I.LastName AS Animators FROM Inhabitant AS I INNER JOIN (SkillActivity AS SA INNER JOIN [Session] AS S ON SA.ActivityName = S.ActivityName) ON I.InHabitantId = S.InHabitantId;': ('07002', '[07002] [Microsoft][Pilote ODBC Microsoft Access] Trop peu de paramètres. 1 attendu. (-3010) (SQLExecDirectW)')

#  3 -	Recommending activities to Solange (Research Question)

In [206]:
# Fix the number of CPU used :
nCPU = os.cpu_count()
pool = ThreadPool(processes=nCPU)

In [207]:
# import the data into a pandas dataframe :
# Import DataSet with parallelization on the different Cpus
path = 'C:/Users/33646/Desktop/M2/EEE/DataBase/Project/activity-rating.csv'
def import_df(file_name):
    df = pd.read_csv(file_name, sep=';')
    return df
df_rating = pool.apply_async(import_df, (path, )).get() 

In [208]:
#select the 3 first columns of the dataframe :
df_rating = df_rating.iloc[:,0:3]
df_rating

,UserId,ActivityId,Rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
1229,597,21,5.0
1230,599,21,4.0
1231,602,21,4.0
1232,603,21,5.0


In [209]:
# We count the number of unique users by counting the unique IDs :
nbunique = df_rating['UserId'].nunique()
print(f"There is {nbunique} unique users.")

There is 385 unique users.


## 1 ) Calculate the similarity (euclidinian distance)

We do it thanks to this website : https://mlforanalytics.com/2018/04/01/euclidean-distance-for-finding-similarity/

In [245]:
df_rating = pd.pivot_table( data=df_rating, 
                        index= ['UserId'], 
                        columns=['ActivityId'], 
                        values='Rating',
                        aggfunc='mean')
df_rating = df_rating.transpose()
#
dict_rating = df_rating.to_dict()


In [251]:
dict_rating


{'UserId': [1,
  5,
  7,
  15,
  17,
  18,
  19,
  21,
  27,
  31,
  32,
  33,
  40,
  43,
  44,
  45,
  46,
  50,
  54,
  57,
  63,
  64,
  66,
  68,
  71,
  73,
  76,
  78,
  82,
  86,
  89,
  90,
  91,
  93,
  96,
  98,
  103,
  107,
  112,
  119,
  121,
  124,
  130,
  132,
  134,
  135,
  137,
  140,
  141,
  144,
  145,
  151,
  153,
  155,
  156,
  159,
  160,
  161,
  166,
  167,
  169,
  171,
  177,
  178,
  179,
  182,
  185,
  186,
  191,
  193,
  200,
  201,
  202,
  206,
  213,
  214,
  216,
  217,
  219,
  220,
  223,
  226,
  229,
  232,
  233,
  234,
  239,
  240,
  247,
  249,
  252,
  254,
  263,
  264,
  266,
  269,
  270,
  273,
  274,
  275,
  276,
  277,
  279,
  280,
  282,
  283,
  288,
  290,
  291,
  292,
  293,
  298,
  304,
  307,
  314,
  322,
  323,
  328,
  330,
  332,
  334,
  336,
  337,
  339,
  341,
  347,
  350,
  353,
  357,
  359,
  364,
  367,
  372,
  373,
  378,
  380,
  381,
  382,
  385,
  389,
  391,
  396,
  399,
  401,
  411,
  412,
  414,


In [241]:
choices= {'John': {'Mango':4.5, 'Banana':3.5, 'Strawberry':4.0, 'Pineapple':4.0},
'Nick': {'Mango':4.0, 'Orange':4.5, 'Banana':3.0, 'Pineapple':4.5},
'Martha': {'Orange':5.0, 'Banana':2.5, 'Strawberry':4.5, 'Apple':3.5},
'Mathew': {'Mango':3.75, 'Strawberry':4.25, 'Apple':3.5, 'Pineapple':3.0}}


,John,Nick,Martha,Mathew
Mango,4.5,4.0,NaN,3.75
Banana,3.5,3.0,2.5,NaN
Strawberry,4.0,NaN,4.5,4.25
Pineapple,4.0,4.5,NaN,3.00
Orange,NaN,4.5,5.0,NaN
Apple,NaN,NaN,3.5,3.50


In [247]:
from math import sqrt
class testClass():
    
    def choice_distance(self, cho, per1, per2):
        #Will set the following dictionary if data is common for two persons
        sample_data={}
        #Above mentioned varibale is an empty dictionary, that is length =0

        for items in cho[per1]:
            if items in cho[per2]:
                sample_data[items]=1
        #Value is being set 1 for those items which are same for both persons

        #If both person does not have any similarity or similar items return 0
                if len(sample_data)==0: 
                    return 0

        #Calculating Euclidean Distance
        final_sum = sum([pow(cho[per1][items]-cho[per2][items],2) for items in cho[per1] if items in cho[per2]])
        return(1/(1+sqrt(final_sum)))
#Value being returned above always lies between 0 and 1
#Value 1 is added to sqrt to prevent 1/0 division and to normaloze result.

In [248]:
ob = testClass()

In [249]:
print(ob.choice_distance(dict_rating, 66, 69))


KeyError: 69

## 2 ) Recommendation algorithm :

We do it thanks to this website :https://mlforanalytics.com/2018/04/04/recommending-items/